In [61]:

import pandas as pd
import psycopg2
import config
import workdays
import datetime
import requests


holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()
vcto_0 = dt.strftime('%d/%m/%Y')
dt_pos = workdays.workday(dt, -1, holidays_br)
dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)




def get_carteira(dt=dt_1):
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)    
    query=f"SELECT dte_data, str_mercado, str_codigo, str_serie, dbl_lote, str_estrategia \
            FROM public.tbl_carteira1 where dte_data='{dt}' and str_estrategia like'%CashCarry%'  and str_fundo='KAPITALO KAPPA MASTER FIM' and str_mesa='Kapitalo 11.1'"

    df =pd.read_sql(query,db_conn_test)
    db_conn_test.close()
    return df

def consulta_ibov():
    r = requests.get('https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwiaW5kZXgiOiJJQk9WIiwic2VnbWVudCI6IjIifQ==', verify=False)
    ibov = pd.DataFrame(r.json()['results'])
    ibov['part']=ibov['part'].str.replace(',', '.').astype(float)
    #reductor = float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

    ibov['theoricalQty'] = ibov['theoricalQty'].apply(lambda x: x.replace('.', '').replace(',', '.')).astype(float)
    ibov['cod']= ibov['cod'].astype(str)
    
    ibov.loc[0,'reductor']= float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

    
    return ibov


df=get_carteira(dt_1)
pos_futuro= df[df['str_mercado']=='Futuro']

pos_futuro = pos_futuro[pos_futuro['str_serie']=="Z22"]

pos_carteira=df[df['str_mercado']=='Acao']
num_contratos=-pos_futuro.iloc[0]['dbl_lote']

display(pos_futuro)

pos_futuro['dbl_lote']=pos_futuro.apply(lambda row: row['dbl_lote']/5 if row['str_codigo']=='WIN' else row['dbl_lote'],axis=1)
pos_futuro['str_codigo']='IND'
    

pos_futuro=pos_futuro.groupby(["str_estrategia",'str_codigo'],as_index=False).agg({'dbl_lote':sum})
pos_futuro

,dte_data,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
98,2022-10-10,Futuro,IND,Z22,1584.0,CashCarry
100,2022-10-10,Futuro,WIN,Z22,2.0,CashCarry
104,2022-10-10,Futuro,IND,Z22,-2056.0,CashCarry5
106,2022-10-10,Futuro,WIN,Z22,-484.0,CashCarry5


,str_estrategia,str_codigo,dbl_lote
0,CashCarry,IND,1584.4
1,CashCarry5,IND,-2152.8


In [62]:
dt = datetime.date.today()
r = requests.get('https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwiaW5kZXgiOiJJQk9WIiwic2VnbWVudCI6IjIifQ==', verify=False)
ibov = pd.DataFrame(r.json()['results']).drop(columns=['segment','partAcum'])
ibov['part']=ibov['part'].str.replace(',', '.').astype(float)
#reductor = float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

ibov['theoricalQty'] = ibov['theoricalQty'].apply(lambda x: x.replace('.', '').replace(',', '.')).astype(float)
ibov['cod']= ibov['cod'].astype(str)

ibov.loc[0,'reductor']= float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

# ibov.to_excel(f"G:\Trading\K11\CashCarry\Carteira Ibovespa\\Carteira_IBOV_{dt.strftime('%Y-%m-%d')}.xlsx")

C:\Python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sistemaswebb3-listados.b3.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [63]:
kappa = pos_futuro['dbl_lote'].sum()

In [60]:
kappa - 482.4

-1055.6

In [64]:
pos_futuro

,str_estrategia,str_codigo,dbl_lote
0,CashCarry,IND,1584.4
1,CashCarry5,IND,-2152.8


In [65]:
df=get_carteira()
ibov=consulta_ibov()
num_contratos=pos_futuro.loc[(pos_futuro['str_estrategia']=="CashCarry")].iloc[0]['dbl_lote']
pos_carteira = pos_carteira[pos_carteira['str_estrategia']=='CashCarry']
display(num_contratos)
redutor=ibov['reductor'].tolist()[0]
ibov=ibov[['cod','theoricalQty']]


pos_carteira['cod']=pos_carteira['str_serie'].apply(lambda x: x.replace(' BZ EQUITY',''))

rebal=pd.merge(pos_carteira,ibov, on='cod',how='inner')

rebal['ideal']= rebal.apply(lambda row: -row['theoricalQty']*num_contratos/redutor, axis=1)

rebal['trade']=round((rebal['ideal']-rebal['dbl_lote'])/100,0)*100
# rebal['trade']=round(rebal['ideal'],0)-rebal['dbl_lote']
rebal=rebal[abs(rebal['trade'])>100]
trade=rebal[['cod','trade']]
# rebal

# trade.to_excel('ajuste_cash.xlsx')
# num_contratos
trade
# trade.to_clipboard(excel=True)





C:\Python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sistemaswebb3-listados.b3.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


1584.4

,cod,trade
14,BRAP4,200.0
42,FLRY3,-200.0
52,JBSS3,-200.0
53,KLBN11,200.0
69,RAIL3,-200.0
75,SBSP3,-200.0


In [29]:
pos_bova=df[df['str_mercado']=="Futuro"]

# roll=pos_bova[pos_bova['str_serie']=='M22']
# pos_bova
# roll
pos_bova['dbl_lote_ind']=pos_bova.apply(lambda row: (row['dbl_lote']/5) if (row['str_codigo']=='WIN') else row['dbl_lote'],axis=1)


# roll['dbl_lote_ind']=roll.apply(lambda row: (row['dbl_lote']/5) if (row['str_codigo']=='WIN') else row['dbl_lote'],axis=1)

pos_bova.groupby(['str_codigo','str_serie','str_estrategia']).sum()

# roll=roll['dbl_lote_ind'].sum()
# roll

# pos_bova['dbl_lote_ind'].sum()

# -950 


# 659 


C:\Python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


dbl_lote  dbl_lote_ind
str_codigo str_serie str_estrategia                        
IND        V22       CashCarry         1258.0        1258.0
                     CashCarry5       -1360.0       -1360.0
WIN        V22       CashCarry         1932.0         386.4
                     CashCarry5       -3964.0        -792.8

In [43]:
pos_bova['dbl_lote_ind'].sum()

-508.39999999999986

In [44]:
pos_bova[pos_bova['str_estrategia']=='CashCarry']['dbl_lote_ind'].sum()

1644.4

In [45]:
## ZETA FIA


    
    
def get_carteira_zeta(dt=dt_1):
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)    
    query=f"SELECT dte_data, str_mercado, str_codigo, str_serie, dbl_lote, str_estrategia \
            FROM public.tbl_carteira1 where dte_data='{dt}' and str_estrategia like'%CashCarry%'  and str_fundo='KAPITALO ZETA MASTER FIA' and str_mesa='Kapitalo 11.1'"

    df =pd.read_sql(query,db_conn_test)
    db_conn_test.close()
    
    return df


In [46]:

df=get_carteira_zeta(dt_1)
pos_futuro= df[df['str_mercado']=='Futuro']


pos_carteira=df[df['str_mercado']=='Acao']
num_contratos=-pos_futuro.iloc[0]['dbl_lote']

df
display(pos_carteira)
pos_carteira.to_clipboard(excel=True, sep=None)
# pos_carteira.to_excel('.xlsx')
pos_futuro['dbl_lote']=pos_futuro.apply(lambda row: row['dbl_lote']/5 if row['str_codigo']=='WIN' else row['dbl_lote'],axis=1)
pos_futuro['str_codigo']='IND'
pos_futuro=pos_futuro.groupby(["str_estrategia",'str_codigo'],as_index=False).agg({'dbl_lote':sum})


,dte_data,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
0,2022-10-06,Acao,BOVA,BOVA11 BZ EQUITY,502941.0,CashCarry5


C:\Python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [47]:
df=get_carteira_zeta(dt_1)
ibov=consulta_ibov()
num_contratos_z=pos_futuro[pos_futuro['str_estrategia']=="CashCarry5"].iloc[0]['dbl_lote']

# redutor=ibov['reductor'].tolist()[0]
# ibov=ibov[['cod','theoricalQty']]


# pos_carteira['cod']=pos_carteira['str_serie'].apply(lambda x: x.replace(' BZ EQUITY',''))

# rebal=pd.merge(pos_carteira,ibov, on='cod',how='inner')

# rebal['ideal']= rebal.apply(lambda row: row['theoricalQty']*num_contratos/redutor, axis=1)

# rebal['trade']=round((rebal['ideal']-rebal['dbl_lote'])/100,0)*100
# rebal=rebal[abs(rebal['trade'])>0]
# trade=rebal[['cod','trade']]

# trade


C:\Python37\lib\site-packages\urllib3\connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sistemaswebb3-listados.b3.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [48]:
df

,dte_data,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
0,2022-10-06,Acao,BOVA,BOVA11 BZ EQUITY,502941.0,CashCarry5
1,2022-10-06,Futuro,IND,V22,-482.0,CashCarry5
2,2022-10-06,Futuro,WIN,V22,-2.0,CashCarry5


In [51]:
num_contratos_z+kappa

-964.8

In [13]:
df

,dte_data,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
0,2022-09-15,Acao,BOVA,BOVA11 BZ EQUITY,502941.0,CashCarry5
1,2022-09-15,Futuro,IND,V22,-482.0,CashCarry5
2,2022-09-15,Futuro,WIN,V22,-2.0,CashCarry5


In [14]:
 query= f"SELECT  * \
            FROM tbl_gerencialajustes where dte_data='{self.dt_1.strftime('%Y-%m-%d')}' and str_mesa ='Kapitalo 11.1' and str_mercado='Futuro' and str_codigo='{name}' and str_fundo='{fundo}' and str_serie='{serie}' "
            db_conn_test = psycopg2.connect(host=acesso.DB_TESTE_HOST, dbname=acesso.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')
            ga=pd.read_sql(query,db_conn_test)

IndentationError: unexpected indent (Temp/ipykernel_21468/3230369956.py, line 3)